# The Weyl scalar $\psi_4$ in Cartesian coordinates
### This tutorial primarily concerns rewriting the arbitrary Weyl scalar code in a format that better facilitates an analytic comparison with Einstein Toolkit's $\text{WeylScal4}$ thorn.
To that end, we use the same physical quantities as the Einstein Toolkit in place of conformal, rescaled quantities. Instead of calling BSSN_RHSs(), which in turn calls reference_metric(), we simply declare the metric $\gamma_{ij}$ and extrinsic curvature $K_{ij}$ (along with various traces, inverses, and determinants) as gridfunctions themselves. 

In [1]:
# Step 1: import all needed modules from NRPy+:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
import reference_metric as rfm
from outputC import *
#import BSSN.BSSN_RHSs as bssn
import sympy as sp


In [2]:
# Step 2: Initialize WeylScalar parameters
thismodule = __name__
# Current option: Approx_QuasiKinnersley = choice made in Baker, Campanelli, and Lousto. PRD 65, 044001 (2002)
par.initialize_param(par.glb_param("char", thismodule, "TetradChoice", "Approx_QuasiKinnersley"))

### Defining the Levi-Civita Symbol

Next we declare a function that defines the rank-3 Levi-Civita symbol, $\epsilon_{ijk}$, applying the algorithm in the [Wikipedia article on the Levi-Civita symbol](https://en.wikipedia.org/wiki/Levi-Civita_symbol), but exploiting properties of computer integer arithmetic as outlined [here](https://codegolf.stackexchange.com/questions/160359/levi-civita-symbol). 

In short, the Levi-Civita symbol $\epsilon_{ijk}$ is set to 1 for even permutations of the indices, -1 for odd permutations, or zero if any indices are repeated.

In [3]:
# Step 3: Define the rank-3 version of the Levi-Civita symbol. Amongst
#         other uses, this is needed for the construction of the approximate 
#         quasi-Kinnersley tetrad.
def define_LeviCivitaSymbol_rank3(DIM=-1):
    if DIM == -1:
        DIM = par.parval_from_str("DIM")

    LeviCivitaSymbol = ixp.zerorank3()

    for i in range(DIM):
        for j in range(DIM):
            for k in range(DIM):
                # From https://codegolf.stackexchange.com/questions/160359/levi-civita-symbol :
                LeviCivitaSymbol[i][j][k] = (i - j) * (j - k) * (k - i) / 2
    return LeviCivitaSymbol

## Defining the Approximate Quasi-Kinnersley Tetrad

To define the Weyl scalars, first a tetrad must be chosen. Below, for compatibility with the [WeylScal4 diagnostic module](https://bitbucket.org/einsteintoolkit/einsteinanalysis/src/master/WeylScal4/), we implement the approximate  quasi-Kinnersley tetrad of the BCL paper.

We begin with the vectors given in eqs. 5.6 and 5.7 of the BCL paper,
\begin{align}
    v_1^a &= [-y,x,0] \\
    v_2^a &= [x,y,z] \\
    v_3^a &= {\rm det}(g)^{1/2} g^{ad} \epsilon_{dbc} v_1^b v_2^c,
\end{align}
and carry out the Gram-Schmidt orthonormalization process. Note that these vectors are initially orthogonal to each other; one is in the $\phi$ direction, one is in $r$, and the third is the cross product of the first two. The vectors $w_i^a$ are placeholders in the code; the final product of the orthonormalization is the vectors $e_i^a$. So,
\begin{align}
e_1^a &= \frac{v_1^a}{\omega_{11}} \\
e_2^a &= \frac{v_2^a - \omega_{12} e_1^a}{\omega_{22}} \\
e_3^a &= \frac{v_3^a - \omega_{13} e_1^a - \omega_{23} e_2^a}{\omega_{33}}, \\
\end{align}
where $\omega_{ij} = v_i^a v_j^b \gamma_{ab}$ needs to be updated between steps (to save resources, we can get away with only calculating components as needed), and uses $e_i^a$ instead of $v_i^a$ if it has been calculated. Recall that $\gamma_{ab}$ was declared as a gridfunction above.

Once we have orthogonal, normalized vectors, we can contruct the tetrad itself, again drawing on eqs. 5.6. We can draw on SymPy's built-in tools for complex numbers to build the complex vectors $m^a$ and $\bar{m}^a$:
\begin{align}
    l^a &= \frac{1}{\sqrt{2}} e_2^a \\
    n^a &= \frac{-1}{\sqrt{2}} e_2^a \\
    m^a &= \frac{1}{\sqrt{2}} (e_3^a + i e_1^a) \\
    \bar{m}^a &= \frac{1}{\sqrt{2}} (e_3^a - i e_1^a)
\end{align}

We will also assume that $n^0 = \frac{1}{\sqrt{2}}$ and that $m^0 = \bar{m}^0 = 0$. This last assumption in particular will significantly reduce the terms needed to find $\psi_4$.

In [4]:
# Step 1: Call BSSN_RHSs. This module computes many different quantities related to the metric,
#         many of which will be essential here. We must first change to our desired coordinate
#         system, however.
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
#bssn.BSSN_RHSs() 
rfm.reference_metric()
# We do not need the barred or hatted quantities calculated when using Cartesian coordinates.
# Instead, we declare the PHYSICAL metric and extrinsic curvature as grid functions.
gammaDD = ixp.register_gridfunctions_for_single_rank2("EVOL","gammaDD", "sym12")
kDD = ixp.register_gridfunctions_for_single_rank2("EVOL","kDD", "sym12")
detgamma = gri.register_gridfunctions("AUX",["detgamma"]) # Should this be its own gridfunction, or the dummydet from below?
gammaUU, dummydet = ixp.symm_matrix_inverter3x3(gammaDD)


# Step 2a: Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Step 2b: Set the coordinate system to Cartesian
x = rfm.xxCart[0]
y = rfm.xxCart[1]
z = rfm.xxCart[2]

# Step 2c: Set which tetrad is used; at the moment, only one supported option
TetradChoice = par.parval_from_str("TetradChoice")

if TetradChoice == "Approx_QuasiKinnersley":
    # Eqs 5.6 in https://arxiv.org/pdf/gr-qc/0104063.pdf
    xmoved = x# - xorig
    ymoved = y# - yorig
    zmoved = z# - zorig

    # Step 3a: Choose 3 orthogonal vectors. Here, we choose one in the azimuthal 
    #          direction, one in the radial direction, and the cross product of the two. 
    # Eqs 5.7
    v1U = ixp.zerorank1()
    v2U = ixp.zerorank1()
    v3U = ixp.zerorank1()
    v1U[0] = -ymoved
    v1U[1] = xmoved# + offset
    v1U[2] = 0
    v2U[0] = xmoved# + offset
    v2U[1] = ymoved
    v2U[2] = zmoved
    LeviCivitaSymbol_rank3 = define_LeviCivitaSymbol_rank3()
    for a in range(DIM):
        for b in range(DIM):
            for c in range(DIM):
                for d in range(DIM):
                    v3U[a] += sp.sqrt(detgamma) * gammaUU[a][d] * LeviCivitaSymbol_rank3[d][b][c] * v1U[b] *v2U[c]

    # Step 3b: Gram-Schmidt orthonormalization of the vectors.
    # The w_i^a vectors here are used to temporarily hold values on the way to the final vectors e_i^a

    #w1U = ixp.zerorank1()
    #for a in range(DIM):
    #    w1U[a] = v1U[a]
    #omega11 = 0
    #for a in range(DIM):
    #    for b in range(DIM):
    #        omega11 += w1U[a] * w1U[b] * gammaDD[a][b]
    #e1U = ixp.zerorank1()
    #for a in range(DIM):
    #    e1U[a] = w1U[a] / sp.sqrt(omega11)

    #omega12 = 0
    #for a in range(DIM):
    #    for b in range(DIM):
    #        omega12 += e1U[a] * v1U[b] * gammaDD[a][b]
    #w2U = ixp.zerorank1()
    #for a in range(DIM):
    #    w2U[a] = v2U[a] - omega12*e1U[a]
    #omega22 = 0
    #for a in range(DIM):
    #    for b in range(DIM):
    #        omega22 += w2U[a] * w2U[b] *gammaDD[a][b]
    #e2U = ixp.zerorank1()
    #for a in range(DIM):
    #    e2U[a] = w2U[a] / sp.sqrt(omega22)

    #omega13 = 0
    #for a in range(DIM):
    #    for b in range(DIM):
    #        omega13 += e1U[a] * v3U[b] * gammaDD[a][b]
    #omega23 = 0
    #for a in range(DIM):
    #    for b in range(DIM):
    #        omega23 += e2U[a] * v3U[b] * gammaDD[a][b]
    #w3U = ixp.zerorank1()
    #for a in range(DIM):
    #    w3U[a] = v3U[a] - omega13*e1U[a] - omega23*e2U[a]
    #omega33 = 0
    #for a in range(DIM):
    #    for b in range(DIM):
    #        omega33 += w3U[a] * w3U[b] * gammaDD[a][b]
    #e3U = ixp.zerorank1()
    #for a in range(DIM):
    #    e3U[a] = w3U[a] / sp.sqrt(omega33)
        
    # Step 3c: Construct the tetrad itself.
    # Eqs. 5.6
    e1U = ixp.declarerank1("e1U")
    e2U = ixp.declarerank1("e2U")
    e3U = ixp.declarerank1("e3U")
    isqrt2 = 1/sp.sqrt(2)
    ltetU = ixp.zerorank1()
    ntetU = ixp.zerorank1()
    mtetU = ixp.zerorank1()
    mtetccU = ixp.zerorank1()
    for i in range(DIM):
        ltetU[i] = isqrt2 * e2U[i]
        ntetU[i] = -isqrt2 * e2U[i]
        mtetU[i] = isqrt2 * (e3U[i] + sp.I*e1U[i])
        mtetccU[i] = isqrt2 * (e3U[i] - sp.I*e1U[i])
    nn = isqrt2

else:
    print("Error: TetradChoice == "+par.parval_from_str("TetradChoice")+" unsupported!")
    exit(1)

## Building the Riemann and extrinsic curvature tensors
Now that we have the tetrad in place, we can contract it with the Weyl tensor to obtain the Weyl scalars. Naturally, we must first construct the Weyl tensor to do that. We will first build the Riemann curvature tensor,
\begin{align}
R_{abcd} = \frac{1}{2} (\gamma_{ad,cb}+\gamma_{bc,da}-\gamma_{ac,bd}-\gamma_{bd,ac}) + \gamma_{je} \Gamma^{j}_{bc}\Gamma^{e}_{ad} - \gamma_{je} \Gamma^{j}_{bd} \Gamma^{e}_{ac}
\end{align}
since several terms in our expression for $\psi_4$ are contractions of this tensor.
To do this, we need second derivatives of the metric tensor, $\gamma_{ab,cd}$, using the finite differencing functionality in NRPy+. 

We also need the extrinsic curvature tensor $K_{ij}$; in the Cartesian case, we simply declared $K_{ij}$ as a gridfunction above.

In [5]:
#Step 4a: Declare and construct the second derivative of the metric.
#gammabarDD_dDD = ixp.zerorank4()
#for i in range(DIM):
#    for j in range(DIM):
#        for k in range(DIM):
#            for l in range(DIM):
#                gammabarDD_dDD[i][j][k][l] = bssn.hDD_dDD[i][j][k][l]*rfm.ReDD[i][j] + \
#                                             bssn.hDD_dD[i][j][k]*rfm.ReDDdD[i][j][l] + \
#                                             bssn.hDD_dD[i][j][l]*rfm.ReDDdD[i][j][k] + \
#                                             bssn.hDD[i][j]*rfm.ReDDdDD[i][j][k][l] + \
#                                             rfm.ghatDDdDD[i][j][k][l]
gammaDD_dD = ixp.declarerank3("gammaDD_dD","sym12")

# Define the Christoffel symbols
GammaUDD = ixp.zerorank3(DIM)
for i in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            for m in range(DIM):
                GammaUDD[i][k][l] += (sp.Rational(1,2))*gammaUU[i][m]*\
                                     (gammaDD_dD[m][k][l] + gammaDD_dD[m][l][k] - gammaDD_dD[k][l][m])


# Step 4b: Declare and construct the Riemann curvature tensor:
gammaDD_dDD = ixp.declarerank4("gammaDD_dDD","sym12_sym34")
RiemannDDDD = ixp.zerorank4()
for a in range(DIM):
    for b in range(DIM):
        for c in range(DIM):
            for d in range(DIM):
                RiemannDDDD[a][b][c][d] = (gammaDD_dDD[a][d][c][b] + \
                                           gammaDD_dDD[b][c][d][a] - \
                                           gammaDD_dDD[a][c][b][d] - \
                                           gammaDD_dDD[b][d][a][c]) / 2
                for e in range(DIM):
                    for j in range(DIM):
                        RiemannDDDD[a][b][c][d] +=  gammaDD[j][e] * GammaUDD[j][b][c] * GammaUDD[e][a][d] - \
                                                    gammaDD[j][e] * GammaUDD[j][b][d] * GammaUDD[e][a][c]

                        
# Step 4c: We also need the extrinsic curvature tensor $K_{ij}$. This can be built from quantities from BSSN_RHSs
# For Cartesian coordinates, we already made this a gridfunction.
#extrinsicKDD = ixp.zerorank2()
#for i in range(DIM):
#    for j in range(DIM):
#        extrinsicKDD[i][j] = (bssn.AbarDD[i][j] + sp.Rational(1,3)*gammaDD[i][j]*bssn.trK)/bssn.exp_m4phi
# We will, however, need to calculate the trace of K seperately:
trK = sp.sympify(0)
for i in range(DIM):
    for j in range(DIM):
        trK += gammaUU[i][j] * kDD[i][j]

# Putting it all together: calculating $\psi_4$
We do not not need to explicitly build the Weyl tensor itself, because the BCL paper shows that, for the Weyl tensor $C_{ijkl}$,
\begin{align}
\psi_4 =&  C_{ijkl} n^i \bar{m}^j n^k \bar{m}^l\\
=& (R_{ijkl} + 2K_{i[k}K_{l]j}) n^i \bar{m}^j n^k \bar{m}^l \\
&- 8 (K_{j[k,l]} + \Gamma^p_{j[k} K_{l]p}) n^{[0} \bar{m}^{j]} n^k \bar{m}^l \\
&+ 4 (R_{jl} - K_{jp} K^p_l + KK_{jl}) n^{[0} \bar{m}^{j]} n^{[0} \bar{m}^{l]}.
\end{align}
Note here the brackets around pairs of indices. This indicates the antisymmetric part of a tensor; that is for some arbitrary tensor $A_{ij}$, $A_{[ij]} = \frac{1}{2}(A_{ij}-A_{ji})$. This applies identically for indices belonging to separate tensors as well as superscripts in place of subscripts.

To make it easier to track the construction of this expression, we will break it down into three parts, by first defining each of the parenthetical terms above separately. That is, let 
\begin{align}
\text{GaussDDDD[i][j][k][l]} =& R_{ijkl} + 2K_{i[k}K_{l]j},
\end{align}

In [6]:
# Step 5: Build the formula for \psi_4.
# Gauss equation: involving the Riemann tensor and extrinsic curvature.
GaussDDDD = ixp.zerorank4()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                GaussDDDD[i][j][k][l] = RiemannDDDD[i][j][k][l] + kDD[i][k]*kDD[l][j] - kDD[i][l]*kDD[k][j]

\begin{align}
\text{CodazziDDD[j][k][l]} =& -2 (K_{j[k,l]} + \Gamma^p_{j[k} K_{l]p}),
\end{align}                                                       

In [7]:
# Codazzi equation: involving partial derivatives of the extrinsic curvature. 
# We will first need to declare derivatives of kDD
kDD_dD = ixp.declarerank3("kDD_dD","sym12")
CodazziDDD = ixp.zerorank3()
for j in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            CodazziDDD[j][k][l] = kDD_dD[j][l][k] - kDD_dD[j][k][l]
            for p in range(DIM):
                CodazziDDD[j][k][l] += GammaUDD[p][j][l]*kDD[k][p] - GammaUDD[p][j][k]*kDD[l][p]

and
\begin{align}
\text{RojoDD[j][l]} = & R_{jl} - K_{jp} K^p_l + KK_{jl} \\
= & \gamma^{pd} R_{jpld} - K_{jp} K^p_l + KK_{jl},
\end{align}

In [8]:
# Another piece. While not associated with any particular equation,
# this is still useful for organizational purposes.
RojoDD = ixp.zerorank2()
for j in range(DIM):
    for l in range(DIM):
        RojoDD[j][l] = trK*kDD[j][l]
        for p in range(DIM):
            for d in range(DIM):
                RojoDD[j][l] += gammaUU[p][d]*RiemannDDDD[j][p][l][d] - kDD[j][p]*gammaUU[p][d]*kDD[d][l]

where these quantities are so named because of their relation to the Gauss-Codazzi equations. Then, we simply contract these with the tetrad we chose earlier to arrive at an expression for $\psi_4$. The barred Christoffel symbols and barred Ricci tensor have already been calculated by BSSN_RHSs.py, so we use those values. So, our expression for $\psi_4$ has become 
\begin{align}
\psi_4 =& (\text{Gauss[i][j][k][l]}) n^i \bar{m}^j n^k \bar{m}^l \\
&+2 (\text{CodazziDDD[j][k][l]}) n^{0} \bar{m}^{j} n^k \bar{m}^l \\
&+ (\text{RojoDD[j][l]}) n^{0} \bar{m}^{j} n^{0} \bar{m}^{l}.
\end{align}

In [9]:
#mtetccU[i] = isqrt2 * (e3U[i] - sp.I*e1U[i])
#full: mtetccU[j] * mtetccU[l]
#real: isqrt2*isqrt2*(e3U[j]*e3U[l]-e1U[j]*e1U[l])
#imag: isqrt2*isqrt2*(-e3U[j]*e1U[l]-e1U[j]*e3U[l])
# Now we can calculate $\psi_4$ itself!
psi4 = sp.sympify(0)
for l in range(DIM):
    for j in range(DIM):
#        psi4 += RojoDD[j][l] * nn * nn * isqrt2*isqrt2*(-e3U[j]*e1U[l]-e1U[j]*e3U[l])
        for k in range(DIM):
#            psi4 += 2 * CodazziDDD[j][k][l] * ntetU[k] * nn * isqrt2*isqrt2*(e3U[j]*e3U[l]-e1U[j]*e1U[l])
            for i in range(DIM):
                psi4 += GaussDDDD[i][j][k][l] * ntetU[i] * ntetU[k] * isqrt2*isqrt2*(e3U[j]*e3U[l]-e1U[j]*e1U[l])

cse_output = sp.cse(psi4,sp.numbered_symbols("tmp"))
#for commonsubexpression in cse_output:
#    print("hello?",commonsubexpression)
for commonsubexpression in cse_output[0]:
    print((str(commonsubexpression[0])+" = "+str(commonsubexpression[1])+";").replace("**","^").replace("_d","d"))
for i,result in enumerate(cse_output[1]):
   print(("psi4rPy= "+str(result)+";").replace("**","^").replace("_d","d"))

tmp0 = gammaDDdDD0011/2;
tmp1 = gammaDDdDD1100/2;
tmp2 = kDD01^2;
tmp3 = kDD00*kDD11;
tmp4 = gammaDD02*gammaDD12;
tmp5 = gammaDD11*gammaDD22;
tmp6 = 2*gammaDD01;
tmp7 = gammaDD12^2;
tmp8 = gammaDD01^2;
tmp9 = gammaDD02^2;
tmp10 = 1/(2*(gammaDD00*tmp5 - gammaDD00*tmp7 - gammaDD11*tmp9 - gammaDD22*tmp8 + tmp4*tmp6));
tmp11 = tmp10*(-gammaDD01*gammaDD22 + tmp4);
tmp12 = tmp10*(tmp5 - tmp7);
tmp13 = -gammaDDdD012 + gammaDDdD021 + gammaDDdD120;
tmp14 = tmp10*(gammaDD01*gammaDD12 - gammaDD02*gammaDD11);
tmp15 = gammaDDdD001*tmp12 + gammaDDdD110*tmp11 + tmp13*tmp14;
tmp16 = gammaDD00*tmp15^2;
tmp17 = tmp10*(gammaDD00*gammaDD22 - tmp9);
tmp18 = tmp10*(-gammaDD00*gammaDD12 + gammaDD01*gammaDD02);
tmp19 = gammaDDdD001*tmp11 + gammaDDdD110*tmp17 + tmp13*tmp18;
tmp20 = gammaDD11*tmp19^2;
tmp21 = tmp10*(gammaDD00*gammaDD11 - tmp8);
tmp22 = gammaDDdD001*tmp14 + gammaDDdD110*tmp18 + tmp13*tmp21;
tmp23 = gammaDD22*tmp22^2;
tmp24 = gammaDD01*tmp15;
tmp25 = 2*tmp19;
tmp26 = tmp24*tmp25;
tmp27 = gammaDD0